In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/Datasets
os.listdir()

/content/gdrive/MyDrive/Datasets


['data', 'logs', 'results']

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import argparse
from keras.applications import InceptionV3
from keras.callbacks import (EarlyStopping,ModelCheckpoint,TensorBoard)
from keras.layers import (AveragePooling2D,Dense,Dropout,Input,Flatten)
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [4]:
tf.get_logger().setLevel('WARNING')

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument('-d', '--data-dir', type=str, default='data/',help="Directory of dataset")
parser.add_argument('-e', '--epochs', type=int, default=30, help="Where to write the new data")
parser.add_argument("-m", "--model", type=str, default="mask_detector.model",help="Path to output face mask detector model")
parser.add_argument('-s', '--size', type=int, default=64,help="Size of input data")
parser.add_argument('-b', '--batch-size', type=int, default=32,help="Batch size of data generator")
parser.add_argument('-l', '--learning-rate', type=float, default=0.0001,help="Learning rate value")

_StoreAction(option_strings=['-l', '--learning-rate'], dest='learning_rate', nargs=None, const=None, default=0.0001, type=<class 'float'>, choices=None, help='Learning rate value', metavar=None)

In [6]:
def VGG16_network(learning_rate, input_shape):
    baseModel = InceptionV3(include_top=False, input_tensor=Input(shape=input_shape))
    for layer in baseModel.layers[:-4]:
        layer.trainable = False

    model = Sequential()
    model.add(baseModel)
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))


    model.compile(loss="binary_crossentropy", metrics=["accuracy"],optimizer=Adam(learning_rate=learning_rate))
    return model

In [7]:
if __name__ == "__main__":

    args,unknown = parser.parse_known_args()

    bs = args.batch_size
    lr = args.learning_rate
    size = (args.size, args.size)
    shape = (args.size,args.size, 3)
    epochs = args.epochs


    train_dir = os.path.join(args.data_dir, 'train')
    test_dir = os.path.join(args.data_dir, 'test')
    valid_dir = os.path.join(args.data_dir, 'validation')

    train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=5, zoom_range=0.2,shear_range=0.2, brightness_range=[0.9, 1.1],horizontal_flip=True)                  
    valid_datagen = ImageDataGenerator(rescale=1./255, rotation_range=5, zoom_range=0.2,shear_range=0.2, brightness_range=[0.9, 1.1],horizontal_flip=True)
    test_datagen  = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_dir, target_size=size, shuffle=True,batch_size=bs, class_mode='binary')
    valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=size, shuffle=True,batch_size=bs, class_mode='binary')
    test_generator  = test_datagen.flow_from_directory(test_dir, target_size=size, shuffle=True,batch_size=bs, class_mode='binary')


    model = VGG16_network(lr, shape)
    model.summary()

    earlystop = EarlyStopping(monitor='val_loss', patience=5, mode='auto')
    tensorboard = TensorBoard(log_dir=os.path.join("logs", 'VGG16'))
    checkpoint = ModelCheckpoint(os.path.join("results", f"{'VGG16'}" + f"-size-{size[0]}" + f"-bs-{bs}" + f"-lr-{lr}.h5"),monitor='val_loss',save_best_only=True, verbose=1)

    history = model.fit(train_generator, epochs=epochs, validation_data=valid_generator,batch_size=bs, callbacks=[earlystop, tensorboard, checkpoint], shuffle=True)
    test_loss, test_accuracy = model.evaluate(test_generator)   
    metrics = pd.DataFrame(history.history)
    

    print('test_loss: ', test_loss)
    print('test_accuracy: ', test_accuracy)
    
    model.save(args.model, save_format="h5")

    
    plt.subplot(211)
    plt.title('Loss')
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()

    plt.subplot(212)
    plt.title('Accuracy')
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='test')
    plt.legend()
    plt.subplots_adjust(left=0.1,bottom=0.1,right=0.9,top=0.9,wspace=0.4,hspace=0.4)
    plt.savefig("plot.png")


Found 3948 images belonging to 2 classes.
Found 1952 images belonging to 2 classes.
Found 260 images belonging to 2 classes.


ValueError: ignored